<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Chapter-10.-Sequence-Hacking,-Hashing,-and-Slicing" data-toc-modified-id="Chapter-10.-Sequence-Hacking,-Hashing,-and-Slicing-1">Chapter 10. Sequence Hacking, Hashing, and Slicing</a></span><ul class="toc-item"><li><span><a href="#Sequence-protocol" data-toc-modified-id="Sequence-protocol-1.1">Sequence protocol</a></span></li><li><span><a href="#How-Slicing-Works" data-toc-modified-id="How-Slicing-Works-1.2">How Slicing Works</a></span><ul class="toc-item"><li><span><a href="#A-Slice-Aware-getitem" data-toc-modified-id="A-Slice-Aware-getitem-1.2.1">A Slice-Aware <strong>getitem</strong></a></span></li></ul></li><li><span><a href="#The-__getattr__-method" data-toc-modified-id="The-__getattr__-method-1.3">The <code>__getattr__</code> method</a></span><ul class="toc-item"><li><span><a href="#The-zip-builtin" data-toc-modified-id="The-zip-builtin-1.3.1">The zip builtin</a></span></li></ul></li></ul></li></ul></div>

# Chapter 10. Sequence Hacking, Hashing, and Slicing

## Sequence protocol

Implement `__len__` and `__getitem__` methods

## How Slicing Works

In [181]:
help(slice.indices)

Help on method_descriptor:

indices(...)
    S.indices(len) -> (start, stop, stride)
    
    Assuming a sequence of length len, calculate the start and stop
    indices, and the stride length of the extended slice described by
    S. Out of bounds indices are clipped in a manner consistent with the
    handling of normal slices.



In [182]:
slice(None, 10, 2).indices(5)

(0, 5, 2)

In [184]:
'ABCDE'[:10:2] == 'ABCDE'[0:5:2]

True

### A Slice-Aware __getitem__

In [384]:
import numbers
import reprlib
import functools  
import operator  


class MySeq():
    typecode = 'd'
    def __init__(self, components):
        self._components = array(self.typecode, components) 
    
    def __iter__(self):
        return iter(self._components) 
    
    def __len__(self):
        return len(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)   
        components = components[components.find('['):-1]  
        class_name = type(self).__name__
        return class_name+'({})'.format(components)

    def __getitem__(self, index):
        cls = type(self)   
        if isinstance(index, slice):   
            return cls(self._components[index])   
        elif isinstance(index, numbers.Integral):   
            return self._components[index]   
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
    
    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)   
        print('shortcut names: {}'.format(cls.shortcut_names))
        if len(name) == 1:   
            pos = cls.shortcut_names.find(name)   
            if 0 <= pos < len(self._components):   
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'   
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:   
            if name not in cls.shortcut_names:   
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():   
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''   
            if error:   
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)  

    def __eq__(self, other):  # 
        return tuple(self) == tuple(other)
    
    def __hash__(self):
        hashes = map(hash, self._components)
        return functools.reduce(operator.xor, hashes)


In [367]:
s1 = MySeq(range(100))
s1[-1]

99.0

In [368]:
s1[1:98]

MySeq([1.0, 2.0, 3.0, 4.0, 5.0, ...])

In [369]:
s1[-1:]

MySeq([99.0])

In [370]:
# does not support multidimensional indexing, 
# so a tuple of indices or slices raises an error.
s1[1,7]

TypeError: MySeq indices must be integers

## The `__getattr__` method
The `__getattr__` method is invoked by the interpreter when attribute lookup fails. 

In simple terms, given the expression `my_obj.x`, Python checks if the my_obj instance has an attribute named x; 

if not, the search goes to the class (`my_obj.__class__`), and then up the inheritance graph. 

If the x attribute is not found, then the `__getattr__` method defined in the class of `my_obj` is called with self and the name of the attribute as a string (e.g., 'x')

In [385]:
s2 = MySeq(range(5))
s2.x

shortcut names: xyzt


0.0

In [390]:
# all attributes are read-only
s2.x = 10

AttributeError: can't set attributes 'a' to 'z' in 'MySeq'

In [387]:
# attributes other than self.components are ignored
s2

shortcut names: xyzt
shortcut names: xyzt
shortcut names: xyzt
shortcut names: xyzt
shortcut names: xyzt
shortcut names: xyzt
shortcut names: xyzt
shortcut names: xyzt
shortcut names: xyzt
shortcut names: xyzt
shortcut names: xyzt
shortcut names: xyzt
shortcut names: xyzt
shortcut names: xyzt
shortcut names: xyzt
shortcut names: xyzt
shortcut names: xyzt
shortcut names: xyzt
shortcut names: xyzt
shortcut names: xyzt
shortcut names: xyzt
shortcut names: xyzt


MySeq([0.0, 1.0, 2.0, 3.0, 4.0])

### The zip builtin 

In [391]:
list(zip(range(3), 'ABC'))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [392]:
list(zip(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3]))  # 

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2)]

In [393]:
# zip_longest uses an optional fillvalue (None by default) to complete 
# missing values so it can generate tuples until the last iterable is exhausted
from itertools import zip_longest
list(zip_longest(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3], fillvalue=-99))

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2), (-99, -99, 3.3)]